<a href="https://colab.research.google.com/github/hhw519/datamining/blob/master/myfirst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
#%%
import  os
import  numpy as np
from    tensorflow import keras
from    tensorflow.keras import layers, losses, optimizers, Sequential

#uploaded = files.upload()
df = pd.read_csv('tran.csv',encoding='cp1252')
target = df.pop('result')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
dataset.batch(4)
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

batchsz = 128 # 批量大小
total_words = 10000 # 词汇表大小N_vocab
max_review_len = 80 # 句子最大长度s，大于的句子部分将截断，小于的将填充
embedding_len = 100 # 词向量特征长度f
# 加载IMDB数据集，此处的数据采用数字编码，一个数字代表一个单词
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)
print(x_train.shape, len(x_train[0]), y_train.shape)
print(x_test.shape, len(x_test[0]), y_test.shape)
#%%
x_train[0]
#%%
# 数字编码表
word_index = keras.datasets.imdb.get_word_index()
# for k,v in word_index.items():
#     print(k,v)
#%%
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3
# 翻转编码表
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(x_train[8])

#%%

# x_train:[b, 80]
# x_test: [b, 80]
# 截断和填充句子，使得等长，此处长句子保留句子后面的部分，短句子在前面填充
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)
# 构建数据集，打散，批量，并丢掉最后一个不够batchsz的batch
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)
print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

#%%

class MyRNN(keras.Model):
    # Cell方式构建多层网络
    def __init__(self, units):
        super(MyRNN, self).__init__() 
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)
        # 构建RNN
        self.rnn = keras.Sequential([
            layers.GRU(units, dropout=0.5, return_sequences=True),
            layers.GRU(units, dropout=0.5)
        ])
        # 构建分类网络，用于将CELL的输出特征进行分类，2分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = Sequential([
        	layers.Dense(32),
        	layers.Dropout(rate=0.5),
        	layers.ReLU(),
        	layers.Dense(1)])

    def call(self, inputs, training=None):
        x = inputs # [b, 80]
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute,[b, 80, 100] => [b, 64]
        x = self.rnn(x)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(x,training)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

def main():
    units = 32 # RNN状态向量长度f
    epochs = 1 # 训练epochs

    model = MyRNN(units)
    # 装配
    model.compile(optimizer = optimizers.Adam(0.001),
                  loss = losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)


if __name__ == '__main__':
    main()



TensorFlow 2.x selected.
2.1.0


NameError: ignored

# 新しいセクション

In [0]:
import numpy as np
import pandas as pd
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import files
#uploaded = files.upload()
df = pd.read_csv('tran.csv',encoding='cp1252')
#df['date'] = pd.Categorical(df['date'])
#df['date'] = df.date.cat.codes
a=df.loc[:,['date','start','high','low','end','trading_volume']]
b=df.loc[:,['trading_volume']]
#np.arrayに変換
npdata=np.array(a.values)
npdata[:10,:]

#nparray=np.reshape(npdata,(data.shape[0],data.shape[1]))
#for idx in range(5,400):
#  np.append([npdata[idx-4:idx]])

#target = df.pop('result')
#dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
#for feat, targ in dataset.take(20):
##  print ('Features: {}, Target: {}'.format(feat, targ))

array([[  43104,     149,     154,     149,     152, 1832900],
       [  43105,     152,     158,     150,     157, 2952500],
       [  43109,     158,     159,     156,     157, 2694500],
       [  43110,     157,     166,     157,     164, 7053300],
       [  43111,     165,     168,     162,     167, 6238800],
       [  43112,     168,     168,     161,     162, 5936700],
       [  43115,     162,     162,     157,     162, 4325100],
       [  43116,     161,     162,     158,     158, 2084400],
       [  43117,     158,     160,     154,     154, 2761500],
       [  43118,     156,     161,     155,     158, 2615000]])